In [181]:
from datetime import date
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
df = pd.read_csv("../Resources/btcjoin.csv", parse_dates=['date'])
df = df.drop(columns=['volume','change', 'low', 'high', 'open'])
df = df.rename(columns={"value": "wallets"})
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
df.dtypes

#https://dcresearch.medium.com/metcalfes-law-and-bitcoin-s-value-2b99c7efd1fa

date       datetime64[ns]
price             float64
wallets             int64
address           float64
mined             float64
dtype: object

In [182]:
df['Metcafe']=df['address']**2
df['value'] = df['Metcafe']/df['mined']
df["value"] = df["value"].map("{:.2f}".format)
df['value']=df['value'].astype("float")
df['networkvalue'] = df["price"] - df["value"]
df.dtypes


date            datetime64[ns]
price                  float64
wallets                  int64
address                float64
mined                  float64
Metcafe                float64
value                  float64
networkvalue           float64
dtype: object

In [183]:
df

,date,price,wallets,address,mined,Metcafe,value,networkvalue
0,2010-09-25,0.1,2,656.0,4109450.00,4.303360e+05,0.10,0.00
1,2010-09-26,0.1,2,444.0,4110750.00,1.971360e+05,0.05,0.05
2,2010-09-27,0.1,2,1070.0,4132150.00,1.144900e+06,0.28,-0.18
3,2010-09-28,0.1,2,1503.0,4133200.00,2.259009e+06,0.55,-0.45
4,2010-09-29,0.1,2,1049.0,4148050.00,1.100401e+06,0.27,-0.17
...,...,...,...,...,...,...,...,...
4381,2022-10-21,19162.6,84879615,924681.0,19185681.25,8.550350e+11,44566.31,-25403.71
4382,2022-10-22,19204.8,84888147,830404.0,19186618.75,6.895708e+11,35940.19,-16735.39
4383,2022-10-23,19571.2,84893809,804140.0,19187600.00,6.466411e+11,33700.99,-14129.79
4384,2022-10-24,19331.5,84902680,919344.0,19188556.25,8.451934e+11,44046.74,-24715.24


In [184]:
df['status'] = df['networkvalue'].apply(lambda x: '1' if x > 0 else '0')
# Create our features
X = df.drop(columns="status")
X = pd.get_dummies(X)

# Create our target
X = df.drop(columns="date")
y = df['status']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, train_size=0.70)


In [185]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

In [186]:
## Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)



In [187]:
print(f'Training Score: {model.score(X_train, y_train)}')
print(f'Testing Score: {model.score(X_test, y_test)}')
print(classification_report(y_test, y_pred))
print(balanced_accuracy_score(y_test, y_pred))

Training Score: 0.9397394136807817
Testing Score: 0.9369300911854104
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      1233
           1       0.00      0.00      0.00        83

    accuracy                           0.94      1316
   macro avg       0.47      0.50      0.48      1316
weighted avg       0.88      0.94      0.91      1316

0.5


c:\Users\16474\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\16474\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\16474\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [188]:
## Oversampling

# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.metrics import classification_report_imbalanced
from sklearn.linear_model import LogisticRegression
from collections import Counter
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [189]:
y_pred = model.predict(X_test)
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
confusion_matrix(y_test, y_pred)
balanced_accuracy_score(y_test, y_pred)



0.5

In [190]:
# Displaying results
print(f'Training Score: {model.score(X_resampled, y_resampled)}')
print(f'Testing Score: {model.score(X_test, y_test)}')
print("Confusion Matrix")
display(cm_df)
print(balanced_accuracy_score(y_test, y_pred))
print("Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Training Score: 0.5
Testing Score: 0.06306990881458967
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,0,1233
Actual 1,0,83


0.5
Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.00      0.00      1.00      0.00      0.00      0.00      1233
          1       0.06      1.00      0.00      0.12      0.00      0.00        83

avg / total       0.00      0.06      0.94      0.01      0.00      0.00      1316



c:\Users\16474\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [191]:
## Undersampling

# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)


# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [192]:
y_pred = model.predict(X_test)
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
confusion_matrix(y_test, y_pred)
balanced_accuracy_score(y_test, y_pred)

0.7218802216163926

In [193]:
# Displaying results
print(f'Training Score: {model.score(X_resampled, y_resampled)}')
print(f'Testing Score: {model.score(X_test, y_test)}')
print("Confusion Matrix")
display(cm_df)
print(balanced_accuracy_score(y_test, y_pred))
print("Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Training Score: 0.9135135135135135
Testing Score: 0.5630699088145896
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1233,0
Actual 1,83,0


0.7218802216163926
Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.54      0.90      0.70      0.70      0.47      1233
          1       0.12      0.90      0.54      0.21      0.70      0.51        83

avg / total       0.93      0.56      0.88      0.67      0.70      0.47      1316



In [194]:
## Over and Under sampling
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)

# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [195]:
y_pred = model.predict(X_test)
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
confusion_matrix(y_test, y_pred)
balanced_accuracy_score(y_test, y_pred)

0.7237514535025749

In [196]:
# Displaying results
print(f'Training Score: {model.score(X_resampled, y_resampled)}')
print(f'Testing Score: {model.score(X_test, y_test)}')
print("Confusion Matrix")
display(cm_df)
print(balanced_accuracy_score(y_test, y_pred))
print("Imbalanced Classification Report")
print(classification_report_imbalanced(y_test, y_pred))

Training Score: 0.6774907749077491
Testing Score: 0.8297872340425532
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1042,191
Actual 1,33,50


0.7237514535025749
Imbalanced Classification Report
                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.85      0.60      0.90      0.71      0.52      1233
          1       0.21      0.60      0.85      0.31      0.71      0.50        83

avg / total       0.92      0.83      0.62      0.87      0.71      0.52      1316

